In [ ]:
import pandas as pd
import os
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models




In [ ]:
for dirname, _, filenames in os.walk(os.getcwd()):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_dir = 'ECG_DATA/train'
test_dir = 'ECG_DATA/test'

In [ ]:
def get_subfolders_and_image_counts(directory):
    subfolders = {}
    for subdir, dirs, files in os.walk(directory):
        if dirs:
            for folder in dirs:
                folder_path = os.path.join(subdir, folder)
                num_images = len([file for file in os.listdir(folder_path) if file.endswith(('.png', '.jpg', '.jpeg'))])
                subfolders[folder] = num_images
    return subfolders

train_subfolders = get_subfolders_and_image_counts(train_dir)
test_subfolders = get_subfolders_and_image_counts(test_dir)

In [ ]:
print("Training Subfolders and Image Counts:")
for folder, count in train_subfolders.items():
    print(f"{folder}: {count} images")

print("\nTest Subfolders and Image Counts:")
for folder, count in test_subfolders.items():
    print(f"{folder}: {count} images")

In [ ]:
# Data Augmentation using Image Data Generator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  
    ,rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'

)

test_datagen = ImageDataGenerator(rescale=1./255,rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
def dataGenerator(train_dir,test_dir):
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),  
        batch_size=34,
        class_mode='categorical',color_mode='grayscale', # Clss_mode = 'categorical 'simplifies that its one hot encoded
        subset='training'  
    )

    validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',color_mode='grayscale',
        subset='validation'  
    )
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),color_mode='grayscale',
        batch_size=32,
        class_mode='categorical'
    )

    return train_generator,validation_generator, test_generator
# print("Class indices:", train_generator.class_indices)

In [ ]:
trainGenerated, validationGenerated, testGenerated = dataGenerator(train_dir,test_dir)

In [ ]:
print("Class indices:", trainGenerated.class_indices)

In [ ]:
def graphs(history): 
    epochs = range(1, len(loss) + 1)
    import matplotlib.pyplot as plt

    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('ModelAccuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'])
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'])
    plt.show()

In [ ]:
def mlpModelFun():
    mlpModel = models.Sequential([
        layers.Flatten(input_shape=(224,224,1)),


        layers.Dense(256,activation='relu'),
        layers.Dropout(0.1),

        layers.Dense(128,activation='relu'),
        layers.Dropout(0.1),

        layers.Dense(64,activation='relu'),
        layers.Dropout(0.1),

        layers.Dense(len(trainGenerated.class_indices))
    ])

    mlpModel.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    trainGenerated = trainGenerated.repeat()  # Add this line to repeat the training data
    validationGenerated = validationGenerated.repeat()
    
    history = mlpModel.fit(
    trainGenerated,
    steps_per_epoch=trainGenerated.samples // trainGenerated.batch_size,
    validation_data=validationGenerated,
    validation_steps=validationGenerated.samples // validationGenerated.batch_size,
    epochs=25  
    )
    return history

In [ ]:
print("Class indices:", trainGenerated.class_indices)

In [ ]:
graphs(mlpModelHistory)